In [ ]:
!pip install -q git+https://github.com/THU-MIG/yolov10.git
!pip install albumentations==1.4
!pip uninstall -y wandb
!mkdir -p /content/weights
!wget -P /content/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt
!wget -P /content/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt
!wget -P /content/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt
!wget -P /content/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt
!wget -P /content/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10x.pt
!wget -P /content/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10l.pt
!ls -lh /content/weights

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.20
    Uninstalling albumentations-1.4.20:
      Successfully uninstalled albumentations-1.4.20
Found existing installation: wandb 0.18.6
Uninstalling wandb-0.18.6:
  Successfully uninstalled wandb-0.18.6
total 408M
-rw-r--r-- 1 root root  80M May 26 15:53 yolov10b.pt
-rw-r--r-- 1 root root 100M May 26 15:53 yolov10l.pt
-rw-r--r-- 1 root root  64M May 26 15:54 yolov10m.pt
-rw-r--r-- 1 root root  11M May 26 15:54 yolov10n.pt
-rw-r--r-- 1 root root  32M May 26 15:54 yolov10s.pt
-rw-r--r-- 1 root root 123M May 26 15:54 yolov10x.pt


In [ ]:
import yaml
from ultralytics import YOLOv10

import random
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import pathlib
%matplotlib inline
from collections import Counter

In [ ]:
!git clone https://github.com/MrAnayDongre/BloodCell-Detection-Datatset.git /content/datasets/BloodCell-Detection-Datatset
!cat datasets/BloodCell-Detection-Datatset/data.yaml

Cloning into '/content/datasets/BloodCell-Detection-Datatset'...
remote: Enumerating objects: 1773, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 1773 (delta 4), reused 83 (delta 2), pack-reused 1686 (from 1)
Receiving objects: 100% (1773/1773), 11.46 MiB | 18.02 MiB/s, done.
Resolving deltas: 100% (6/6), done.
train: ../train/images
val: ../valid/images

nc: 3
names: ['Platelets', 'RBC', 'WBC']

In [ ]:
# Load the YAML file
with open('datasets/BloodCell-Detection-Datatset/data.yaml', 'r') as f:
    data_yaml = yaml.safe_load(f)

# update YAML with absolute path to kaggle data. You must use absolute path, relative path won't work
data_yaml['train'] = 'BloodCell-Detection-Datatset/train/images'
data_yaml['val'] = 'BloodCell-Detection-Datatset/valid/images'

# write to disk
with open('data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

In [ ]:
data_dir = Path("datasets/BloodCell-Detection-Datatset")
images_dir = data_dir / "train" / "images"
labels_dir = data_dir / "train" / "labels"

class_names = data_yaml['names']

# Read the image file paths and annotations
image_paths = list(images_dir.glob("*.jpg"))
label_paths = sorted(labels_dir.glob("*.txt"))

In [ ]:
baseline_model = YOLOv10('/content/weights/yolov10x.pt')
params = {
    'data':'data.yaml',
    'imgsz':416,
    'epochs':150,
    'optimizer':'Adam',
    'pretrained':True,
    'lr0':0.1,
    'lrf': 0.00001,
    'cos_lr': True
}
results = baseline_model.train(**params)

New https://pypi.org/project/ultralytics/8.3.29 available 😃 Update with 'pip install -U ultralytics'


/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/weights/yolov10m.pt, data=data.yaml, epochs=150, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, val_period=1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

100%|██████████| 755k/755k [00:00<00:00, 26.7MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1     78720  ultralytics.nn.modules.block.SCDown          [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1    228672  ultralytics

...

    150/150      4.22G      1.153     0.8128      1.221      1.224     0.7883      1.235        184        416: 100%|██████████| 48/48 [00:16<00:00,  2.91it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.31it/s]

                   all         73        967      0.846      0.897      0.899      0.614



150 epochs completed in 0.833 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 33.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 33.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10m summary (fused): 369 layers, 16453858 parameters, 0 gradients, 63.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.63it/s]


                   all         73        967      0.834      0.907      0.902      0.624
             Platelets         73         76      0.825      0.895      0.872      0.488
                   RBC         73        819      0.757      0.825      0.866      0.609
                   WBC         73         72      0.919          1      0.968      0.774
Speed: 0.1ms preprocess, 10.2ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!yolo task=detect mode=predict save=True source=/content/datasets/BloodCell-Detection-Datatset/valid/images model=runs/detect/train/weights/best.pt

/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")
Ultralytics YO

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!zip -r /content/drive/MyDrive/newtrainm.zip runs/detect/train
!zip -r /content/drive/MyDrive/newpredictm.zip runs/detect/predict

  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/train_batch6722.jpg (deflated 10%)
  adding: runs/detect/train/train_batch2.jpg (deflated 3%)
  adding: runs/detect/train/val_batch2_pred.jpg (deflated 8%)
  adding: runs/detect/train/val_batch1_labels.jpg (deflated 9%)
  adding: runs/detect/train/train_batch1.jpg (deflated 5%)
  adding: runs/detect/train/R_curve.png (deflated 11%)
  adding: runs/detect/train/results.png (deflated 8%)
  adding: runs/detect/train/val_batch0_labels.jpg (deflated 9%)
  adding: runs/detect/train/F1_curve.png (deflated 10%)
  adding: runs/detect/train/confusion_matrix.png (deflated 31%)
  adding: runs/detect/train/P_curve.png (deflated 11%)
  adding: runs/detect/train/events.out.tfevents.1731503540.d979c8e2d349.252.0 (deflated 88%)
  adding: runs/detect/train/val_batch1_pred.jpg (deflated 7%)
  adding: runs/detect/train/args.yaml (deflated 52%)
  adding: runs/detect/train/val_batch0_pred.jpg (deflated 6%)
  adding: runs/detect/train/confus